Convert rds files from R to python

In [1]:
suppressMessages({
    library(Seurat, quietly = T)
    library(SeuratDisk, quietly = T)
})

Specify directories:

In [2]:
data_path = '/nobackup/users/hmbaghda/scLEMBAS/analysis'

In [3]:
directory_names <- c(#data_path, 
                     #file.path(data_path, 'raw'), file.path(data_path, 'raw', 'immune_dictionary'), 
                     file.path(data_path, 'interim'), file.path(data_path, 'interim', 'immune_dictionary_h5ad'),
                     file.path(data_path, 'processed'), file.path(data_path, 'figures'))

for (directory_name in directory_names){
    if (!file.exists(directory_name)) {
      dir.create(directory_name)
    }
}

Convert from Seurat to Anndata:

In [4]:
seurat_to_h5ad <- function(file_name, from_path, dest_path, rm_h5seurat = TRUE){
    suppressMessages({
        suppressWarnings({
            so<-readRDS(file.path(from_path, file_name))
            so<-UpdateSeuratObject(so)

            # remove scale.data assay slot
            # otherwise, SeuratDisk will only store scaled and log-normalized data, and not raw data
            n_genes <- nrow(so[["RNA"]]@data)
            empty_matrix <- matrix(numeric(0), nrow = n_genes, ncol = 0)
            so[["RNA"]]@scale.data <- empty_matrix
            
            so@assays$RNA[['scale.data']]<-NULL 
            fn<-file.path(dest_path, sub("\\.RDS$", ".h5Seurat", file_name)) # a file name (no path) that ends with .RDS
            SeuratDisk::SaveH5Seurat(so, filename = fn)
            SeuratDisk::Convert(fn, dest = "h5ad", assay = 'RNA')
            if (rm_h5seurat){
                na = file.remove(fn)
            }
        })
    })
}

In [5]:
from_path<-file.path(data_path, 'raw/immune_dictionary/')
dest_path<-file.path(data_path, 'interim', 'immune_dictionary_h5ad')
file_names<-list.files(path = from_path, pattern = "\\.RDS$", recursive = TRUE)
for (file_name in file_names){
    if (!(file.exists(file.path(dest_path, sub("\\.RDS$", ".h5ad", file_name))))){
        seurat_to_h5ad(file_name, from_path, dest_path)
    }
}